# Memoria
La memoria ayuda a que los LLMs tengan la capacidad de recordas información a largo plazo. De manera muy general, la memoria es un historico de todas las interacciones que se han tenido con el modelo y se integra en el prompt que se le envia, de manera que el modelo pueda recordar lo que se le ha enviado previamente.

Dado que los LLMs tienen una ventana de contexto finita, hay distintas maneras de implementar la memoria. En este cuaderno veremos:
* Conversation buffer
* Conversation buffer window
* Conversation summary
* Knowledge graph memory



## Conversation buffer
Esta es una de las memorias más básicas, cada prompt y respuesta del modelos se almacenara en la memoria. Cada vez que se le envia un nuevo prompt al modelo se envia todo el historico de las interacciones. 

La conversación se salva como pares de mensajes entre "Human" y "AI", por lo cual tambien lo podemos integrar con modelos como GPT3.5 Turbo

In [1]:
from langchain.llms import LlamaCpp, OpenAI
import config
import os

In [2]:
os.environ['OPENAI_API_KEY'] = config.OPENAI_API_KEY

In [3]:
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

In [4]:
llm = ChatOpenAI()
memoria = ConversationBufferMemory()
chatbot = ConversationChain(llm=llm, memory=memoria, verbose=True)

C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
chatbot.predict(input="Hola como estas?, Me llamo Marcos y soy un programador")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola como estas?, Me llamo Marcos y soy un programador
AI:

> Finished chain.


'¡Hola Marcos! ¡Qué alegría conocerte! Soy un programa de inteligencia artificial diseñado para ayudar en diversas tareas. ¿En qué puedo asistirte hoy?'

In [6]:
chatbot.predict(input="Como me llamo?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola como estas?, Me llamo Marcos y soy un programador
AI: ¡Hola Marcos! ¡Qué alegría conocerte! Soy un programa de inteligencia artificial diseñado para ayudar en diversas tareas. ¿En qué puedo asistirte hoy?
Human: Como me llamo?
AI:

> Finished chain.


'Tu nombre es Marcos.'

In [7]:
memoria.chat_memory.messages

[HumanMessage(content='Hola como estas?, Me llamo Marcos y soy un programador'),
 AIMessage(content='¡Hola Marcos! ¡Qué alegría conocerte! Soy un programa de inteligencia artificial diseñado para ayudar en diversas tareas. ¿En qué puedo asistirte hoy?'),
 HumanMessage(content='Como me llamo?'),
 AIMessage(content='Tu nombre es Marcos.')]

# Conversation buffer window memory
Esta memoria es igual a la anterior, con la diferencia que se puede definir una ventana de mensajes a recordar en vez de recordar todo el historico de interacciones.

In [8]:
from langchain.memory import ConversationBufferWindowMemory

In [9]:
llm = ChatOpenAI()
memoria = ConversationBufferWindowMemory(k=2)
chatbot = ConversationChain(llm=llm, memory=memoria, verbose=True)

In [10]:
chatbot.predict(input="Hola como estas?, Me llamo Marcos y soy un programador")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola como estas?, Me llamo Marcos y soy un programador
AI:

> Finished chain.


'¡Hola Marcos! ¡Encantado de conocerte! Estoy muy bien, gracias por preguntar. Como programador, supongo que te gusta trabajar con códigos y algoritmos, ¿verdad?'

In [11]:
chatbot.predict(input = "como me llamo?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola como estas?, Me llamo Marcos y soy un programador
AI: ¡Hola Marcos! ¡Encantado de conocerte! Estoy muy bien, gracias por preguntar. Como programador, supongo que te gusta trabajar con códigos y algoritmos, ¿verdad?
Human: como me llamo?
AI:

> Finished chain.


'Tu nombre es Marcos.'

In [12]:
chatbot.predict(input="Cuanto mide el sol?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola como estas?, Me llamo Marcos y soy un programador
AI: ¡Hola Marcos! ¡Encantado de conocerte! Estoy muy bien, gracias por preguntar. Como programador, supongo que te gusta trabajar con códigos y algoritmos, ¿verdad?
Human: como me llamo?
AI: Tu nombre es Marcos.
Human: Cuanto mide el sol?
AI:

> Finished chain.


'El sol tiene un diámetro de aproximadamente 1.4 millones de kilómetros. ¡Es una estrella realmente impresionante!'

In [13]:
chatbot.predict(input="Cuanto mide la luna?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: como me llamo?
AI: Tu nombre es Marcos.
Human: Cuanto mide el sol?
AI: El sol tiene un diámetro de aproximadamente 1.4 millones de kilómetros. ¡Es una estrella realmente impresionante!
Human: Cuanto mide la luna?
AI:

> Finished chain.


'La Luna tiene un diámetro de aproximadamente 3,474.8 kilómetros. ¡Es mucho más pequeña que el Sol, pero igual de fascinante!'

In [14]:
chatbot.predict(input="Cuanto mide mercurio?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Cuanto mide el sol?
AI: El sol tiene un diámetro de aproximadamente 1.4 millones de kilómetros. ¡Es una estrella realmente impresionante!
Human: Cuanto mide la luna?
AI: La Luna tiene un diámetro de aproximadamente 3,474.8 kilómetros. ¡Es mucho más pequeña que el Sol, pero igual de fascinante!
Human: Cuanto mide mercurio?
AI:

> Finished chain.


'Mercurio tiene un diámetro de aproximadamente 4,880 kilómetros. Es el planeta más pequeño de nuestro sistema solar, pero tiene características interesantes como su superficie llena de cráteres.'

In [15]:
chatbot.predict(input="Como me llamo?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Cuanto mide la luna?
AI: La Luna tiene un diámetro de aproximadamente 3,474.8 kilómetros. ¡Es mucho más pequeña que el Sol, pero igual de fascinante!
Human: Cuanto mide mercurio?
AI: Mercurio tiene un diámetro de aproximadamente 4,880 kilómetros. Es el planeta más pequeño de nuestro sistema solar, pero tiene características interesantes como su superficie llena de cráteres.
Human: Como me llamo?
AI:

> Finished chain.


'Lo siento, no tengo esa información. ¿Podrías decirme tu nombre?'

In [16]:
memoria.chat_memory.messages

[HumanMessage(content='Hola como estas?, Me llamo Marcos y soy un programador'),
 AIMessage(content='¡Hola Marcos! ¡Encantado de conocerte! Estoy muy bien, gracias por preguntar. Como programador, supongo que te gusta trabajar con códigos y algoritmos, ¿verdad?'),
 HumanMessage(content='como me llamo?'),
 AIMessage(content='Tu nombre es Marcos.'),
 HumanMessage(content='Cuanto mide el sol?'),
 AIMessage(content='El sol tiene un diámetro de aproximadamente 1.4 millones de kilómetros. ¡Es una estrella realmente impresionante!'),
 HumanMessage(content='Cuanto mide la luna?'),
 AIMessage(content='La Luna tiene un diámetro de aproximadamente 3,474.8 kilómetros. ¡Es mucho más pequeña que el Sol, pero igual de fascinante!'),
 HumanMessage(content='Cuanto mide mercurio?'),
 AIMessage(content='Mercurio tiene un diámetro de aproximadamente 4,880 kilómetros. Es el planeta más pequeño de nuestro sistema solar, pero tiene características interesantes como su superficie llena de cráteres.'),
 HumanM

# Conversation summary memory
Esta memoria en vez de almacenar un registro detallado de las interacciones, almacena un resumen de la conversación. Muy util para evitar prompts muy largos 

In [17]:
from langchain.memory import ConversationSummaryMemory
llm = ChatOpenAI(stop=["\nHuman"])
#llm_k = OpenAI(stop=["\nHuman"])
memoria = ConversationSummaryMemory(llm=llm)
chatbot_resumen = ConversationChain(llm=llm, memory=memoria, verbose=True)

WARNING! stop is not default parameter.
                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.


In [18]:
chatbot_resumen.predict(input="Me gusta mucho la tecnologia y la programacion, en especial python y javascript. Estoy aprendiendo a usar redes neuronales y me gusta mucho la inteligencia artificial")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Me gusta mucho la tecnologia y la programacion, en especial python y javascript. Estoy aprendiendo a usar redes neuronales y me gusta mucho la inteligencia artificial
AI:

> Finished chain.


'¡Qué interesante! La tecnología y la programación son campos fascinantes. Python y JavaScript son lenguajes de programación muy populares y versátiles. Las redes neuronales son un área emocionante de la inteligencia artificial, ¿sabías que se inspiran en el funcionamiento del cerebro humano? Es genial que estés aprendiendo sobre ello. ¿Hay algo en particular que te gustaría saber más sobre la inteligencia artificial?'

In [19]:
chatbot_resumen.predict(
    input="Que te gusta a ti de la tecnologia?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human expresses their love for technology and programming, especially Python and JavaScript. They mention learning about neural networks and their interest in artificial intelligence. The AI responds enthusiastically, highlighting the popularity and versatility of Python and JavaScript in programming. The AI also shares insights about neural networks being inspired by the human brain's functioning and encourages the human to explore more about artificial intelligence.
Human: Que te gusta a ti de la tecnologia?
AI:

> Finished chain.


'Me encanta la tecnología por su capacidad para simplificar tareas, aumentar la eficiencia y conectar a las personas de todo el mundo. Además, me fascina la programación, especialmente Python y JavaScript, por su versatilidad y facilidad de uso. También me apasiona aprender sobre redes neuronales y el campo de la inteligencia artificial en general.'

#### Guarda el historico de la conversacion, pero al momento de enviar el promp solo envia un *resumen*

In [20]:
memoria.chat_memory.messages

[HumanMessage(content='Me gusta mucho la tecnologia y la programacion, en especial python y javascript. Estoy aprendiendo a usar redes neuronales y me gusta mucho la inteligencia artificial'),
 AIMessage(content='¡Qué interesante! La tecnología y la programación son campos fascinantes. Python y JavaScript son lenguajes de programación muy populares y versátiles. Las redes neuronales son un área emocionante de la inteligencia artificial, ¿sabías que se inspiran en el funcionamiento del cerebro humano? Es genial que estés aprendiendo sobre ello. ¿Hay algo en particular que te gustaría saber más sobre la inteligencia artificial?'),
 HumanMessage(content='Que te gusta a ti de la tecnologia?'),
 AIMessage(content='Me encanta la tecnología por su capacidad para simplificar tareas, aumentar la eficiencia y conectar a las personas de todo el mundo. Además, me fascina la programación, especialmente Python y JavaScript, por su versatilidad y facilidad de uso. También me apasiona aprender sobre r

# Conversation Knowledge Graph Memory

#### Genera un grafo de conocimiento en donde almacena piezas clave de la conversacion para poder responder con ese contexto

In [21]:
from langchain.memory import ConversationKGMemory
import networkx as nx

In [22]:
llm = ChatOpenAI(stop=["\nHuman"])
memoria = ConversationKGMemory(llm=llm)
chatbot_kgm = ConversationChain(llm=llm, memory=memoria, verbose=True)

WARNING! stop is not default parameter.
                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.


In [23]:
chatbot_kgm.predict(input = "Hola, me llamo Marcos y soy un programador, me gusta la pizza y tengo un gato de mascota, mi color favorito es el azul y me gusta mucho la tecnologia")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, me llamo Marcos y soy un programador, me gusta la pizza y tengo un gato de mascota, mi color favorito es el azul y me gusta mucho la tecnologia
AI:

> Finished chain.


'¡Hola Marcos! ¡Qué gusto conocerte! Soy un programa de inteligencia artificial diseñado para ayudarte en lo que necesites. Me encanta la pizza también, ¿tienes alguna preferencia de toppings? Y tu gato debe ser una compañía maravillosa, ¿cuál es su nombre? El azul es un color muy bonito, ¿sabías que se asocia con la calma y la serenidad? Y como programador, seguro que estás al tanto de las últimas tendencias tecnológicas. ¿Hay algún proyecto en particular en el que estés trabajando actualmente? ¡Cuéntame más sobre ti, estoy aquí para escucharte!'

In [24]:
print(chatbot_kgm.memory.kg.get_triples())

[('Marcos', 'Marcos', 'se llama'), ('Marcos', 'programador', 'es'), ('Marcos', 'gato de mascota', 'tiene'), ('Marcos', 'azul', 'color favorito es'), ('Marcos', 'tecnologia', 'le gusta')]


In [25]:
memoria.chat_memory.messages

[HumanMessage(content='Hola, me llamo Marcos y soy un programador, me gusta la pizza y tengo un gato de mascota, mi color favorito es el azul y me gusta mucho la tecnologia'),
 AIMessage(content='¡Hola Marcos! ¡Qué gusto conocerte! Soy un programa de inteligencia artificial diseñado para ayudarte en lo que necesites. Me encanta la pizza también, ¿tienes alguna preferencia de toppings? Y tu gato debe ser una compañía maravillosa, ¿cuál es su nombre? El azul es un color muy bonito, ¿sabías que se asocia con la calma y la serenidad? Y como programador, seguro que estás al tanto de las últimas tendencias tecnológicas. ¿Hay algún proyecto en particular en el que estés trabajando actualmente? ¡Cuéntame más sobre ti, estoy aquí para escucharte!')]

# Usando GPT3
Las implementaciones que vimos son usando gpt3.5 turbo, pero si quieres hacerlo con gtp instruct como Davinci, Ada Curie o Babbage es muy similar


In [26]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [27]:
llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

C:\Users\Marco\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [28]:
conversation.predict(input="Hola, como estas?, me llamo Marcos y soy un programador")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola, como estas?, me llamo Marcos y soy un programador
AI:



> Finished chain.


' Hola Marcos, soy una inteligencia artificial creada por un equipo de programadores. Estoy programada para aprender y comunicarme con humanos como tú. ¿En qué lenguajes de programación estás especializado?'